In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlxtend.evaluate import bias_variance_decomp

# Reading in the data
from spotify_data import *
# data = pd.read_csv("dataset.csv")
data

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113995,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113996,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113997,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [24]:
# genre_dict = {}
# for genre in np.unique(spotify["track_genre"]):
#     genre_dict[genre] = genre # spotify[spotify["track_genre"] == genre]
#     print(genre_dict[genre])
# np.unique(spotify["track_genre"])
genre_groups = {genre: group for genre, group in data.groupby('track_genre')}
groupedgenre_groups = {genre: group for genre, group in grouped_data.groupby('track_genre')}

np.unique(data["track_genre"])
np.unique(grouped_data['track_genre'])

array(['classical', 'electronic', 'folk', 'hip-hop', 'jazz', 'metal',
       'misc', 'pop', 'rock', 'world'], dtype=object)

In [25]:
numeric_spotify = data.select_dtypes(include=[np.number])
numeric_spotify = numeric_spotify.loc[:, ~numeric_spotify.columns.isin(["mode", "key", "time_signature"])]
# numeric_spotify

# numeric_pop = pop_df.select_dtypes(include=[np.number])
# numeric_pop = numeric_pop.loc[:, ~numeric_pop.columns.isin(["mode", "key", "time_signature"])]
# numeric_pop = numeric_pop.drop(columns="Unnamed: 0")
# # numeric_pop

numeric_spotify

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,73,230666,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.7150,87.917
1,55,149610,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.2670,77.489
2,57,210826,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.1200,76.332
3,71,201933,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.1430,181.740
4,82,198853,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.1670,119.949
...,...,...,...,...,...,...,...,...,...,...,...
113994,21,384999,0.172,0.2350,-16.393,0.0422,0.6400,0.928000,0.0863,0.0339,125.995
113995,22,385000,0.174,0.1170,-18.318,0.0401,0.9940,0.976000,0.1050,0.0350,85.239
113996,22,271466,0.629,0.3290,-10.895,0.0420,0.8670,0.000000,0.0839,0.7430,132.378
113997,41,283893,0.587,0.5060,-10.889,0.0297,0.3810,0.000000,0.2700,0.4130,135.960


In [26]:
numeric_dict = {
    'popularity' : 0,
    'duration_ms' : 0,
    'danceability' : 0,
    'energy' : 0,
    'loudness' : 0,
    'speechiness' : 0,
    'acousticness' : 0,
    'instrumentalness' : 0,
    'liveness' : 0,
    'valence' : 0,
    'tempo' : 0
}

In [28]:
# Modeling regression of energy versus popularity
# We are doing a linear and quadratic regression analysis.

for genre in genre_groups:
    df = genre_groups.get(genre, pd.DataFrame())
    numeric_df = df.select_dtypes(include=[np.number])
    numeric_df = numeric_df.loc[:, ~numeric_df.columns.isin(["mode", "key", "time_signature"])]

    linear_reg = LinearRegression()

        # Select features
    selector = SequentialFeatureSelector(
            linear_reg,
            n_features_to_select='auto',
            direction='forward',
            scoring='r2',
            cv = 5
        )
    
    max_train_r2 = 0
    max_validation_r2 = 0
    max_response = ''
    
    for x in numeric_df.columns:
        random_seed = 42
        response = x
        # Splitting the data
        # First split: separate out 20% for the test set
        train_val, test = train_test_split(numeric_df, test_size=0.2, random_state=random_seed)

        # Second split: separate remaining 80% into 60% training and 40% validation
        train, val = train_test_split(train_val, test_size=0.25, random_state=random_seed)  # 0.25 * 0.8 = 0.2

        # Reshape the data to fit the model
        X_train = train.drop(columns=response)
        y = train[response]


        selector.fit(X_train, y)
        selected_features = selector.get_feature_names_out(X_train.columns)
        # print("Selected Features: ", selected_features)

        # Transform data sets
        X = selector.transform(X_train)

        X_test = selector.transform(test.drop(columns=response))
        y_test = test[response]

        X_val = selector.transform(val.drop(columns=response))
        y_val = val[response]


        # Fit the Linear Regression model
        linear_reg.fit(X, y)

        # Predict values of y based on the model
        y_pred = linear_reg.predict(X)

        # Calculating Evaluation Metrics:
        y_val_pred = linear_reg.predict(X_val) # Predict on validation set

        # Calculate metrics for the training set
        train_mse = mean_squared_error(y, y_pred) # Mean squared error
        train_rmse = np.sqrt(train_mse) # Root mean squared error
        train_mae = mean_absolute_error(y, y_pred) # Mean average error
        train_mad = np.mean(np.abs(y - y_pred))  # Mean absolute deviation
        train_r2 = r2_score(y, y_pred) # R^2 (coefficient of determination)

        # Calculate metrics for the validation set
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        val_mad = np.mean(np.abs(y_val - y_val_pred))
        val_r2 = r2_score(y_val, y_val_pred)

        # Print Results:
        print(f"{genre} {response} Training MSE: {train_mse}, rMSE: {train_rmse}, MAE: {train_mae}, MAD: {train_mad}, R²: {train_r2}")
        print(f"{genre} {response} Validation MSE: {val_mse}, rMSE: {val_rmse}, MAE: {val_mae}, MAD: {val_mad}, R²: {val_r2}")

        if val_r2 > max_validation_r2:
            max_train_r2 = train_r2
            max_validation_r2 = val_r2
            max_response = x

        # bias variance
        avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
            linear_reg,
            X, y.values,
            X_test, y_test.values,
            loss='mse',
            random_seed=123
        )

        print(f"{genre} {response} Loss, variance, and bias: {avg_expected_loss}, {avg_bias}, {avg_var}")
    print(f"Response Variable with the Highest Validation R^2 for {genre}: {max_response}")
    print(f"Train R^2 for {max_response} in {genre}: {max_train_r2}")
    print(f"Validation R^2 for {max_response} in {genre}: {max_validation_r2}")
    numeric_dict[max_response]+=1

print(numeric_dict)


acoustic popularity Training MSE: 240.1301887882887, rMSE: 15.496134640234922, MAE: 11.938610575897624, MAD: 11.938610575897624, R²: 0.08202801149089911
acoustic popularity Validation MSE: 308.2152415457153, rMSE: 17.556059966453613, MAE: 13.063805517662317, MAD: 13.063805517662317, R²: 0.033764403328190085
acoustic popularity Loss, variance, and bias: 264.9212703269596, 262.7144247551956, 2.2068455717641005
acoustic duration_ms Training MSE: 2334862607.9080443, rMSE: 48320.4160568599, MAE: 37180.19783090932, MAD: 37180.19783090932, R²: 0.19322722623809263
acoustic duration_ms Validation MSE: 3164494809.262861, rMSE: 56253.84261775244, MAE: 40015.50441524246, MAD: 40015.50441524246, R²: 0.06634947269482872
acoustic duration_ms Loss, variance, and bias: 3063600128.744078, 3028701412.797077, 34898715.94700096
acoustic danceability Training MSE: 0.012414960421764605, rMSE: 0.11142244128435082, MAE: 0.08970008412696967, MAD: 0.08970008412696967, R²: 0.15643634524190686
acoustic danceabilit

In [30]:
# Modeling regression of energy versus popularity
# We are doing a linear and quadratic regression analysis.

numeric_dict1 = {
    'popularity' : 0,
    'duration_ms' : 0,
    'danceability' : 0,
    'energy' : 0,
    'loudness' : 0,
    'speechiness' : 0,
    'acousticness' : 0,
    'instrumentalness' : 0,
    'liveness' : 0,
    'valence' : 0,
    'tempo' : 0
}

for genre in groupedgenre_groups:
    df = groupedgenre_groups.get(genre, pd.DataFrame())
    numeric_df = df.select_dtypes(include=[np.number])
    numeric_df = numeric_df.loc[:, ~numeric_df.columns.isin(["mode", "key", "time_signature"])]

    linear_reg = LinearRegression()

        # Select features
    selector = SequentialFeatureSelector(
            linear_reg,
            n_features_to_select='auto',
            direction='forward',
            scoring='r2',
            cv = 5
        )
    
    max_train_r2 = 0
    max_validation_r2 = 0
    max_response = ''
    
    for x in numeric_df.columns:
        random_seed = 42
        response = x
        # Splitting the data
        # First split: separate out 20% for the test set
        train_val, test = train_test_split(numeric_df, test_size=0.2, random_state=random_seed)

        # Second split: separate remaining 80% into 60% training and 40% validation
        train, val = train_test_split(train_val, test_size=0.25, random_state=random_seed)  # 0.25 * 0.8 = 0.2

        # Reshape the data to fit the model
        X_train = train.drop(columns=response)
        y = train[response]


        selector.fit(X_train, y)
        selected_features = selector.get_feature_names_out(X_train.columns)
        # print("Selected Features: ", selected_features)

        # Transform data sets
        X = selector.transform(X_train)

        X_test = selector.transform(test.drop(columns=response))
        y_test = test[response]

        X_val = selector.transform(val.drop(columns=response))
        y_val = val[response]


        # Fit the Linear Regression model
        linear_reg.fit(X, y)

        # Predict values of y based on the model
        y_pred = linear_reg.predict(X)

        # Calculating Evaluation Metrics:
        y_val_pred = linear_reg.predict(X_val) # Predict on validation set

        # Calculate metrics for the training set
        train_mse = mean_squared_error(y, y_pred) # Mean squared error
        train_rmse = np.sqrt(train_mse) # Root mean squared error
        train_mae = mean_absolute_error(y, y_pred) # Mean average error
        train_mad = np.mean(np.abs(y - y_pred))  # Mean absolute deviation
        train_r2 = r2_score(y, y_pred) # R^2 (coefficient of determination)

        # Calculate metrics for the validation set
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        val_mad = np.mean(np.abs(y_val - y_val_pred))
        val_r2 = r2_score(y_val, y_val_pred)

        # Print Results:
        print(f"{genre} {response} Training MSE: {train_mse}, rMSE: {train_rmse}, MAE: {train_mae}, MAD: {train_mad}, R²: {train_r2}")
        print(f"{genre} {response} Validation MSE: {val_mse}, rMSE: {val_rmse}, MAE: {val_mae}, MAD: {val_mad}, R²: {val_r2}")

        if val_r2 > max_validation_r2:
            max_train_r2 = train_r2
            max_validation_r2 = val_r2
            max_response = x

        # bias variance
        avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
            linear_reg,
            X, y.values,
            X_test, y_test.values,
            loss='mse',
            random_seed=123
        )

        print(f"{genre} {response} Loss, variance, and bias: {avg_expected_loss}, {avg_bias}, {avg_var}")
    print(f"Response Variable with the Highest Validation R^2 for {genre}: {max_response}")
    print(f"Train R^2 for {max_response} in {genre}: {max_train_r2}")
    print(f"Validation R^2 for {max_response} in {genre}: {max_validation_r2}")
    numeric_dict1[max_response]+=1

print(numeric_dict1)


classical popularity Training MSE: 441.6984854829015, rMSE: 21.01662402677703, MAE: 17.45099700685927, MAD: 17.45099700685927, R²: 0.12431870021016622
classical popularity Validation MSE: 415.13874354901236, rMSE: 20.374953829371304, MAE: 16.811575626285244, MAD: 16.811575626285244, R²: 0.14469914200820977
classical popularity Loss, variance, and bias: 417.77503924327596, 416.1422599095012, 1.6327793337749397
classical duration_ms Training MSE: 21261526577.52304, rMSE: 145813.32784599302, MAE: 78980.98428829464, MAD: 78980.98428829464, R²: 0.07762035033995118
classical duration_ms Validation MSE: 22040142077.636295, rMSE: 148459.2269871977, MAE: 79365.42759205696, MAD: 79365.42759205696, R²: 0.06465957597208849
classical duration_ms Loss, variance, and bias: 14565201103.196692, 14489441135.648531, 75759967.54816002
classical danceability Training MSE: 0.013966919173843849, rMSE: 0.11818172098020847, MAE: 0.09557503063276233, MAD: 0.09557503063276233, R²: 0.38706766373625623
classical d

In [31]:
# Define the degree of the polynomial
degree = 2

# Create a pipeline for Polynomial Regression
poly_reg = make_pipeline(
    PolynomialFeatures(degree=degree),  # Generates polynomial features
    LinearRegression()                 # Fits a linear model to the transformed features
)

In [32]:
numeric_dict2 = {
    'popularity' : 0,
    'duration_ms' : 0,
    'danceability' : 0,
    'energy' : 0,
    'loudness' : 0,
    'speechiness' : 0,
    'acousticness' : 0,
    'instrumentalness' : 0,
    'liveness' : 0,
    'valence' : 0,
    'tempo' : 0
}

for genre in genre_groups:
    df = genre_groups.get(genre, pd.DataFrame())
    numeric_df = df.select_dtypes(include=[np.number])
    numeric_df = numeric_df.loc[:, ~numeric_df.columns.isin(["mode", "key", "time_signature"])]

        # Select features
    selector = SequentialFeatureSelector(
            linear_reg,
            n_features_to_select='auto',
            direction='forward',
            scoring='r2',
            cv = 5
        )
    
    max_train_r2 = 0
    max_validation_r2 = 0
    max_response = ''
    
    for x in numeric_df.columns:
        random_seed = 42
        response = x
        # Splitting the data
        # First split: separate out 20% for the test set
        train_val, test = train_test_split(numeric_df, test_size=0.2, random_state=random_seed)

        # Second split: separate remaining 80% into 60% training and 40% validation
        train, val = train_test_split(train_val, test_size=0.25, random_state=random_seed)  # 0.25 * 0.8 = 0.2

        # Reshape the data to fit the model
        X_train = train.drop(columns=response)
        y = train[response]


        selector.fit(X_train, y)
        selected_features = selector.get_feature_names_out(X_train.columns)
        # print("Selected Features: ", selected_features)

        # Transform data sets
        X = selector.transform(X_train)

        X_test = selector.transform(test.drop(columns=response))
        y_test = test[response]

        X_val = selector.transform(val.drop(columns=response))
        y_val = val[response]


        # Fit the Linear Regression model
        poly_reg.fit(X, y)

        # Predict values of y based on the model
        y_pred = poly_reg.predict(X)

        # Calculating Evaluation Metrics:
        y_val_pred = poly_reg.predict(X_val) # Predict on validation set

        # Calculate metrics for the training set
        train_mse = mean_squared_error(y, y_pred) # Mean squared error
        train_rmse = np.sqrt(train_mse) # Root mean squared error
        train_mae = mean_absolute_error(y, y_pred) # Mean average error
        train_mad = np.mean(np.abs(y - y_pred))  # Mean absolute deviation
        train_r2 = r2_score(y, y_pred) # R^2 (coefficient of determination)

        # Calculate metrics for the validation set
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        val_mad = np.mean(np.abs(y_val - y_val_pred))
        val_r2 = r2_score(y_val, y_val_pred)

        # Print Results:
        print(f"{genre} {response} Training MSE: {train_mse}, rMSE: {train_rmse}, MAE: {train_mae}, MAD: {train_mad}, R²: {train_r2}")
        print(f"{genre} {response} Validation MSE: {val_mse}, rMSE: {val_rmse}, MAE: {val_mae}, MAD: {val_mad}, R²: {val_r2}")

        if val_r2 > max_validation_r2:
            max_train_r2 = train_r2
            max_validation_r2 = val_r2
            max_response = x

        # bias variance
        avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
            linear_reg,
            X, y.values,
            X_test, y_test.values,
            loss='mse',
            random_seed=123
        )

        print(f"{genre} {response} Loss, variance, and bias: {avg_expected_loss}, {avg_bias}, {avg_var}")
    print(f"Response Variable with the Highest Validation R^2 for {genre}: {max_response}")
    print(f"Train R^2 for {max_response} in {genre}: {max_train_r2}")
    print(f"Validation R^2 for {max_response} in {genre}: {max_validation_r2}")
    numeric_dict2[max_response]+=1

print(numeric_dict2)

acoustic popularity Training MSE: 238.36334924002847, rMSE: 15.43902034586484, MAE: 11.82710415440354, MAD: 11.82710415440354, R²: 0.08878230265969056
acoustic popularity Validation MSE: 307.9668771079297, rMSE: 17.548985073443127, MAE: 13.020023325311787, MAD: 13.020023325311787, R²: 0.03454301038062635
acoustic popularity Loss, variance, and bias: 264.9212703269596, 262.7144247551956, 2.2068455717641005
acoustic duration_ms Training MSE: 2218123169.954381, rMSE: 47096.95499662776, MAE: 36108.270706506286, MAD: 36108.270706506286, R²: 0.23356458906462185
acoustic duration_ms Validation MSE: 4964752876.611097, rMSE: 70461.00252346043, MAE: 42569.36859108882, MAD: 42569.36859108882, R²: -0.4647975176383927
acoustic duration_ms Loss, variance, and bias: 3063600128.744078, 3028701412.797077, 34898715.94700096
acoustic danceability Training MSE: 0.00968638823350577, rMSE: 0.09841945048366085, MAE: 0.07813410628386003, MAD: 0.07813410628386003, R²: 0.34183559334291336
acoustic danceability 

In [33]:
# Modeling regression of energy versus popularity
# We are doing a linear and quadratic regression analysis.

numeric_dict3 = {
    'popularity' : 0,
    'duration_ms' : 0,
    'danceability' : 0,
    'energy' : 0,
    'loudness' : 0,
    'speechiness' : 0,
    'acousticness' : 0,
    'instrumentalness' : 0,
    'liveness' : 0,
    'valence' : 0,
    'tempo' : 0
}

for genre in groupedgenre_groups:
    df = groupedgenre_groups.get(genre, pd.DataFrame())
    numeric_df = df.select_dtypes(include=[np.number])
    numeric_df = numeric_df.loc[:, ~numeric_df.columns.isin(["mode", "key", "time_signature"])]

    linear_reg = LinearRegression()

        # Select features
    selector = SequentialFeatureSelector(
            linear_reg,
            n_features_to_select='auto',
            direction='forward',
            scoring='r2',
            cv = 5
        )
    
    max_train_r2 = 0
    max_validation_r2 = 0
    max_response = ''
    
    for x in numeric_df.columns:
        random_seed = 42
        response = x
        # Splitting the data
        # First split: separate out 20% for the test set
        train_val, test = train_test_split(numeric_df, test_size=0.2, random_state=random_seed)

        # Second split: separate remaining 80% into 60% training and 40% validation
        train, val = train_test_split(train_val, test_size=0.25, random_state=random_seed)  # 0.25 * 0.8 = 0.2

        # Reshape the data to fit the model
        X_train = train.drop(columns=response)
        y = train[response]


        selector.fit(X_train, y)
        selected_features = selector.get_feature_names_out(X_train.columns)
        # print("Selected Features: ", selected_features)

        # Transform data sets
        X = selector.transform(X_train)

        X_test = selector.transform(test.drop(columns=response))
        y_test = test[response]

        X_val = selector.transform(val.drop(columns=response))
        y_val = val[response]


        # Fit the Linear Regression model
        linear_reg.fit(X, y)

        # Predict values of y based on the model
        y_pred = linear_reg.predict(X)

        # Calculating Evaluation Metrics:
        y_val_pred = linear_reg.predict(X_val) # Predict on validation set

        # Calculate metrics for the training set
        train_mse = mean_squared_error(y, y_pred) # Mean squared error
        train_rmse = np.sqrt(train_mse) # Root mean squared error
        train_mae = mean_absolute_error(y, y_pred) # Mean average error
        train_mad = np.mean(np.abs(y - y_pred))  # Mean absolute deviation
        train_r2 = r2_score(y, y_pred) # R^2 (coefficient of determination)

        # Calculate metrics for the validation set
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        val_mad = np.mean(np.abs(y_val - y_val_pred))
        val_r2 = r2_score(y_val, y_val_pred)

        # Print Results:
        print(f"{genre} {response} Training MSE: {train_mse}, rMSE: {train_rmse}, MAE: {train_mae}, MAD: {train_mad}, R²: {train_r2}")
        print(f"{genre} {response} Validation MSE: {val_mse}, rMSE: {val_rmse}, MAE: {val_mae}, MAD: {val_mad}, R²: {val_r2}")

        if val_r2 > max_validation_r2:
            max_train_r2 = train_r2
            max_validation_r2 = val_r2
            max_response = x

        # bias variance
        avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
            linear_reg,
            X, y.values,
            X_test, y_test.values,
            loss='mse',
            random_seed=123
        )

        print(f"{genre} {response} Loss, variance, and bias: {avg_expected_loss}, {avg_bias}, {avg_var}")
    print(f"Response Variable with the Highest Validation R^2 for {genre}: {max_response}")
    print(f"Train R^2 for {max_response} in {genre}: {max_train_r2}")
    print(f"Validation R^2 for {max_response} in {genre}: {max_validation_r2}")
    numeric_dict3[max_response]+=1

print(numeric_dict3)

classical popularity Training MSE: 441.6984854829015, rMSE: 21.01662402677703, MAE: 17.45099700685927, MAD: 17.45099700685927, R²: 0.12431870021016622
classical popularity Validation MSE: 415.13874354901236, rMSE: 20.374953829371304, MAE: 16.811575626285244, MAD: 16.811575626285244, R²: 0.14469914200820977
classical popularity Loss, variance, and bias: 417.77503924327596, 416.1422599095012, 1.6327793337749397
classical duration_ms Training MSE: 21261526577.52304, rMSE: 145813.32784599302, MAE: 78980.98428829464, MAD: 78980.98428829464, R²: 0.07762035033995118
classical duration_ms Validation MSE: 22040142077.636295, rMSE: 148459.2269871977, MAE: 79365.42759205696, MAD: 79365.42759205696, R²: 0.06465957597208849
classical duration_ms Loss, variance, and bias: 14565201103.196692, 14489441135.648531, 75759967.54816002
classical danceability Training MSE: 0.013966919173843849, rMSE: 0.11818172098020847, MAE: 0.09557503063276233, MAD: 0.09557503063276233, R²: 0.38706766373625623
classical d